# Hospital appointment management system

In [5]:
import os #python built in modeule to check wheather file exists or not.
from datetime import datetime   #this modules allow to check date and time formate

# Doctor class
class Doctor:
    def __init__(e, doc_id, full_name, specialization, timing): #constructor
        e.doc_id = doc_id
        e.full_name = full_name
        e.specialization = specialization
        e.timing = timing

    def format_record(e):
        return f"{e.doc_id},{e.full_name},{e.specialization},{e.timing}\n"
        # staticmethod defines a method that can be called without creating an object of the class
    @staticmethod
    def load_doctors():
        doctor_list = [] #doctor list to add doctors
        if os.path.isfile("doctors.txt"): #check if the file exists
            with open("doctors.txt", "r") as doc_file: #open and read file
                for record in doc_file:
                    parts = record.strip().split(",") #remove whitespace and newline
                    if len(parts) == 4: #checks that there are 4 items in constructor
                        doctor_list.append(Doctor(*parts))
        return doctor_list
    @staticmethod
    def setup_default_doctors():
        if not os.path.exists("doctors.txt") or os.path.getsize("doctors.txt") == 0:
            default_docs = [
                ("1", "Dr. Sajjad", "Cardiologist", "9 AM - 5 PM"),
                ("2", "Dr. Ali", "Neurologist", "9 AM - 5 PM"),
                ("3", "Dr. Paul Winchel", "Surgeon", "9 AM - 5 PM"),
                ("4", "Dr. Huma", "Dermatologist", "9 AM - 5 PM"),
            ]   # a list of default doctor records to be saved into the file

            with open("doctors.txt", "w") as doc_file:
             for entry in default_docs:
                      doc_file.write(",".join(entry) + "\n")

# Patient Record Manager
class Patient:
    def __init__(self, pid, pname, page):
        self.pid = pid
        self.pname = pname
        self.page = page

    def save_patient(self):
        with open("patients.txt", "a") as p_file:
            p_file.write(f"{self.pid},{self.pname},{self.page}\n")

    @staticmethod
    def fetch_all_patients():
        patient_data = []
        if os.path.exists("patients.txt"):
            with open("patients.txt", "r") as p_file:
                for line in p_file:
                    parts = line.strip().split(",")
                    if len(parts) == 3:
                        patient_data.append(Patient(*parts))
        return patient_data


# Appointments
class Appointment:
    def __init__(e, appt_id, doc_name, pat_name, pat_age, appt_time):
        e.appt_id = appt_id
        e.doc_name = doc_name
        e.pat_name = pat_name
        e.pat_age = pat_age
        e.appt_time = appt_time

    def record_appointment(e):
        with open("appointments.txt", "a") as a_file:
            a_file.write(f"{e.appt_id},{e.doc_name},{e.pat_name},{e.pat_age},{e.appt_time}\n") #appoinment formate: 1,Dr. Ali,faisal,30,2025-04-21 11:00

    @staticmethod
    def check_slot(doc_name, appt_time):
        if not os.path.isfile("appointments.txt"):
            return False
        with open("appointments.txt", "r") as file:
            for line in file:
                _, d_name, _, _, d_time = line.strip().split(",")
                if d_name == doc_name and d_time == appt_time:
                    return True
        return False

    @staticmethod
    def remove_appointment(appt_id):
        if not os.path.exists("appointments.txt"):
            return False
        updated_records = []
        was_found = False
        with open("appointments.txt", "r") as file:
            for line in file:
                if not line.startswith(appt_id + ","):
                    updated_records.append(line)
                else:
                    was_found = True
        with open("appointments.txt", "w") as file:
            file.writelines(updated_records)
        return was_found

    @staticmethod
    def display_appointments():
        if not os.path.exists("appointments.txt"): #checks if the file exists
            print("no appointments available.")
            return
        print("\n== Appointments List ==")
        with open("appointments.txt", "r") as file:
            for line in file:
                parts = line.strip().split(",") #remove extra space and divide line inti list
                if len(parts) == 5:
                    print(f"Appointment #: {parts[0]}")
                    print(f"Doctor: {parts[1]}")
                    print(f"Patient: {parts[2]} ({parts[3]} years)")
                    print(f"Scheduled Time: {parts[4]}\n")

# unique id will be generate
def get_next_id(file_name):
    if not os.path.exists(file_name):
        return "1"
    with open(file_name, "r") as file:
        return str(len(file.readlines()) + 1)

# main menu
def main_menu():
    Doctor.setup_default_doctors()
#condition statement
    while True:
        print("\n=== Clinic Appointment System ===")
        print("1. add new patient")
        print("2. make appointment")
        print("3. cancel appointment")
        print("4. show doctor List")
        print("5. show all Appointments")
        print("6. quit")

        option = input("choose any option : ")

        if option == '1':
            pname = input("pateint name: ")
            page = input("pateint age: ")
            pid = get_next_id("patients.txt")
            new_patient = Patient(pid, pname, page)
            new_patient.save_patient()
            print(f"pateint has been registered with id: {pid}")

        elif option == '2':
            doctors = Doctor.load_doctors()
            print("\n== Available Doctors ==")
            for doc in doctors:
              print(f"{doc.full_name} - {doc.specialization} ({doc.timing})")

            selected_doc = input("Enter doctor name: ")
            matched = [d for d in doctors if d.full_name.lower() == selected_doc.lower()]
            if not matched:
                print("doctor is not availble.")
                continue

            pname = input("Patient Name: ")
            page = input("Patient Age: ")
            appt_time = input("Enter Date & Time (YYYY-MM-DD HH:MM): ")

            try:
                datetime.strptime(appt_time, "%Y-%m-%d %H:%M")
                if Appointment.check_slot(selected_doc, appt_time):
                    print("is time kisi or ki booking hai.")
                else:
                    appt_id = get_next_id("appointments.txt")
                    new_appt = Appointment(appt_id, selected_doc, pname, page, appt_time)
                    new_appt.record_appointment()
                    print(f"Appointment confirmed with ID: {appt_id}")
            except ValueError:
                print("Invalid format. Use YYYY-MM-DD HH:MM.")

        elif option == '3':
            appt_id = input("Enter Appointment ID to cancel: ")
            if Appointment.remove_appointment(appt_id):
                print("Appointment successfully cancelled.")
            else:
                print("Appointment ID not found.")

        elif option == '4':
            doc_list = Doctor.load_doctors()
            if not doc_list:
                print("No doctor data found.")
            else:
                print("\n=== Doctor Information ===")
                for doc in doc_list:
                    print(f"ID: {doc.doc_id} | Name: {doc.full_name} | Field: {doc.specialization} | Timing: {doc.timing}")

        elif option == '5':
            Appointment.display_appointments()

        elif option == '6':
            print("Thanks for using the system. Goodbye!")
            break

        else:
            print("Ap kahein or ilaj karwaye, shukriya")

# Start program
if __name__ == "__main__":
    main_menu()



=== Clinic Appointment System ===
1. add new patient
2. make appointment
3. cancel appointment
4. show doctor List
5. show all Appointments
6. quit
choose any option : 4

=== Doctor Information ===
ID: 1 | Name: Dr. Sajjad | Field: Cardiologist | Timing: 9 AM - 5 PM
ID: 2 | Name: Dr. Ali | Field: Neurologist | Timing: 9 AM - 5 PM
ID: 3 | Name: Dr. Paul Winchel | Field: Surgeon | Timing: 9 AM - 5 PM
ID: 4 | Name: Dr. Huma | Field: Dermatologist | Timing: 9 AM - 5 PM

=== Clinic Appointment System ===
1. add new patient
2. make appointment
3. cancel appointment
4. show doctor List
5. show all Appointments
6. quit
choose any option : 5

== Appointments List ==
Appointment #: 1
Doctor: Dr. Paul winchel
Patient: khan (23 years)
Scheduled Time: 2025-04-22 04:20


=== Clinic Appointment System ===
1. add new patient
2. make appointment
3. cancel appointment
4. show doctor List
5. show all Appointments
6. quit
choose any option : 2

== Available Doctors ==
Dr. Sajjad - Cardiologist (9 AM - 5 